# Web Scraping del sito Kijiji



## Import delle librerie


In [1]:
import requests
import bs4
import csv
from tqdm import tqdm_notebook as tqdm
import json
import pprint
import re

## Definizione url di riferimento

Per semplificare il progetto la ricerca riguarderà tutti gli annunci trovati filtrando per regione Lombardia e prodotto iPhone.


In [2]:
webpage = f"https://www.kijiji.it/annunci-lombardia/iphone/?p=1"

Cerco tutti gli annunci nella pagina:

In [3]:
response = requests.get(webpage)
doc = bs4.BeautifulSoup(response.text)
annunci = doc.find_all('li', class_="result")

C:\Users\carla.molteni\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\carla.molteni\AppData\Local\Continuum\anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [4]:
len(annunci)

26

Ci sono 26 annunci nella pagina!

Test del codice per lo scarico delle prime informazioni nella pagina di ricerca degli annunci:

In [5]:
annunci_list = []
for annuncio in annunci:
    try:
        titolo = annuncio.select("h3.title > a")[0].text.replace("\n", "").strip()
    except:
        continue
    url = annuncio["data-href"]
    annuncio_id = annuncio["data-id"]
    try:
        prezzo = annuncio.select("div.item-content > h4.price")[0].text.replace("\n", "").replace("€","").strip()
    except:
        prezzo = "N/A"
    try:
        data = annuncio.select("div.item-content > p.timestamp")[0].text
    except:
        data = "N/A"
    try:
        paese = annuncio.select("div.item-content > p.locale")[0].text
    except:
        paese = "N/A"
    annunci_list.append({'id': annuncio_id, 'titolo': titolo, 'link': url, 
                         'prezzo': prezzo, 'data': data, 'paese' :paese})
    
print(annunci_list[0])

{'id': '138705039', 'titolo': 'IPhone 8 (2018) Nero - 64GB', 'link': 'https://www.kijiji.it/annunci/cellulari-e-accessori/milano-annunci-maciachini-zara-niguarda/iphone-8-2018-nero-64gb/138705039', 'prezzo': '400', 'data': 'Ieri, 11:43', 'paese': 'Maciachini / Zara / Niguarda'}


Ok, ora posso definire la funzione per il parsing generico degli annunci:

In [6]:

def parse_annuncio(annuncio):
    try:
        titolo = annuncio.select("h3.title > a")[0].text.replace("\n", "").strip()
    except:
        return None
    url = annuncio["data-href"]
    annuncio_id = annuncio["data-id"]
    try:
        prezzo = annuncio.select("div.item-content > h4.price")[0].text.replace("\n", "").replace("€","").strip()
    except:
        prezzo = "N/A"
    try:
        data = annuncio.select("div.item-content > p.timestamp")[0].text
    except:
        data = "N/A"
    try:
        paese = annuncio.select("div.item-content > p.locale")[0].text
    except:
        paese = "N/A"
    return {'id': annuncio_id, 'titolo': titolo, 'link': url, 
                         'prezzo': prezzo, 'data': data, 'paese' :paese}


In un ciclo for gestisco il download degli annunci per ogni pagina:

In [7]:
annunci_list = []
for num in tqdm(range(1,11)):
    webpage = f"https://www.kijiji.it/annunci-lombardia/iphone/?p={num}"
    response = requests.get(webpage)
    doc = bs4.BeautifulSoup(response.text)
    annunci = doc.find_all('li', class_="result")
    for annuncio in annunci:
        annunci_list.append(parse_annuncio(annuncio))

print(len(annunci_list))
        

A Jupyter Widget

C:\Users\carla.molteni\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\carla.molteni\AppData\Local\Continuum\anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))



260


## Come terminare lo scraping?

Problema: *quando mi fermo?*
- cerco all'interno della pagina il numero totale di pagine


In [9]:
firstpage = f"https://www.kijiji.it/annunci-lombardia/iphone/?p=1"
firstresponse = requests.get(firstpage)
firstdoc = bs4.BeautifulSoup(firstresponse.text)
pagecount = int(firstdoc.select("#pagination > div > a.last-page")[0].text)
print(pagecount)

C:\Users\carla.molteni\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\carla.molteni\AppData\Local\Continuum\anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


47


In [10]:
annunci_list = []
annunci_letti = 0
num = 0
for num in tqdm(range(1,pagecount)):
    num = num + 1
    webpage = f"https://www.kijiji.it/annunci-lombardia/iphone/?p={num}"
    response = requests.get(webpage)
    doc = bs4.BeautifulSoup(response.text)
    annunci = doc.find_all('li', class_="result")
    for annuncio in annunci:
        try:
            annuncio_parsed = parse_annuncio(annuncio)
            if annuncio_parsed is not None:
                annunci_list.append(annuncio_parsed)
        except:
            pass
    annunci_letti = len(annunci_list)

A Jupyter Widget

C:\Users\carla.molteni\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\carla.molteni\AppData\Local\Continuum\anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


## Pandas
Utilizzo la libreria Pandas per salvare i risultati in un file csv:

In [11]:
import pandas as pd
ds_annunci = pd.DataFrame(annunci_list)
ds_annunci.set_index("id")
ds_annunci.head()

,data,id,link,paese,prezzo,titolo
0,"Ieri, 11:43",138705039,https://www.kijiji.it/annunci/cellulari-e-acce...,Maciachini / Zara / Niguarda,400,IPhone 8 (2018) Nero - 64GB
1,"Ieri, 09:04",138808297,https://www.kijiji.it/annunci/cellulari-e-acce...,Como,890,Iphone xs max 512GB
2,"Ieri, 11:43",138705039,https://www.kijiji.it/annunci/cellulari-e-acce...,Maciachini / Zara / Niguarda,400,IPhone 8 (2018) Nero - 64GB
3,"Ieri, 10:40",139317911,https://www.kijiji.it/annunci/cellulari-e-acce...,Cunardo,40,Stampa foto Iphone e Android
4,"Ieri, 10:40",137693462,https://www.kijiji.it/annunci/cerco-lavoro-ser...,Mantova,Contatta l'utente,Riparatore telefono iPhone samsug Huwei originale


In [12]:
ds_annunci.to_csv("./kijiji_annunci.csv")

## Le pagine degli annunci ###
Ora l'obiettivo è scaricare i dettagli dei singoli annunci e le foto.
# Geocoding
Per avere una maggiore precisione riguardo all'informazione geografica degli annunci, richiamo il servizio di geocoding in modo da ottenere una geolocalizzazione più strutturata.


In [14]:
import json
# apro il file csv
import pandas as pd
ds_annunci = pd.read_csv("./kijiji_annunci.csv",encoding = "ISO-8859-1", index_col="id")
ds_annunci.head()

,Unnamed: 0,data,link,paese,prezzo,titolo
id,,,,,,
138705039,0,"Ieri, 11:43",https://www.kijiji.it/annunci/cellulari-e-acce...,Maciachini / Zara / Niguarda,400,IPhone 8 (2018) Nero - 64GB
138808297,1,"Ieri, 09:04",https://www.kijiji.it/annunci/cellulari-e-acce...,Como,890,Iphone xs max 512GB
138705039,2,"Ieri, 11:43",https://www.kijiji.it/annunci/cellulari-e-acce...,Maciachini / Zara / Niguarda,400,IPhone 8 (2018) Nero - 64GB
139317911,3,"Ieri, 10:40",https://www.kijiji.it/annunci/cellulari-e-acce...,Cunardo,40,Stampa foto Iphone e Android
137693462,4,"Ieri, 10:40",https://www.kijiji.it/annunci/cerco-lavoro-ser...,Mantova,Contatta l'utente,Riparatore telefono iPhone samsug Huwei originale


Per ogni annuncio vado a richiamare il link e a procedere con lo scarico dalla pagina di dettaglio:

In [96]:
dettagli = []
for annuncio_id, annuncio in tqdm(ds_annunci.head().iterrows(), total=ds_annunci.head().shape[0]):
    dettagli_row = {}
    link = annuncio["link"]
    print(link)
    response = requests.get(link)
    doc = bs4.BeautifulSoup(response.text)
    try:
        descrizione = doc.select("article.vip__carousel > div.box__content > div > p.vip__text-description")[0].text.replace('\n', ' ').strip()
    except:
        continue
    try:
        nome_venditore = doc.select("article.user > div.media__body > div.title")[0].text.replace('\n', ' ').strip()
    except:
        nome_venditore = "N/A"
    try: 
        phone = doc.select("h3.modal-phone__text")[0].text.replace('\n', ' ').strip().replace("+", "00")
    except:
        phone = 'N/A'
    try: 
        userId = doc.select("div.user__bottom > a")[0]["href"].split("/")[-1]
    except:
        userId = 'N/A'
    id_ad = doc.select("li.breadcrumbs__leaf > strong")[0].text.replace('\n', ' ').strip()
    try:
        location = doc.select("div.vip__location > div > div > span")[0].text.replace('\n', ' ').strip()
    except: 
        location1 = doc.select("div.vip__location")[0]
        location =location1.select("div > span")[0].text
    print(location)
    dettagli_row.update({'descrizione': descrizione,  'nome_venditore': nome_venditore,
                        'phone': phone, 'userId':userId, 'id_ad':id_ad, 'location': location})
    details = doc.select("section.vip__details > dl")
    new_table = pd.DataFrame(columns=range(0,2), index = [0]) # I know the size 
    row_marker = 0
    for row in details:
        #column_marker = 0
        label = row.find_all('dt')[0].text.replace('\n', ' ').strip()
        value = row.find_all('dd')[0].text.replace('\n', ' ').strip()
        dettagli_row.update({label:value})
    ## geocoding
    lat = ""
    lon = ""
    try:
        key = "T4eqDjtnpzWsfeMBZgKAqKobvcICurpU"
        geocode_url = f"http://www.mapquestapi.com/geocoding/v1/address?key={key}&location={dettagli_row.get('location')}"
        response = requests.get(geocode_url)
        geo = json.loads(response.text)
        lat = geo['results'][0]['locations'][0]['latLng']['lat']
        lon = geo['results'][0]['locations'][0]['latLng']['lng']
    except:
        pass
    #scarico le immagini
    docImg = doc.select("section.vip__body > article.vip__carousel > div.carousel-slide ")[0]
    imgs = docImg.find_all("img")
    i = 0
    for img in imgs:
        src = img['src']
        print(src)
        img_file = requests.get(src, stream=True)
        if img_file.status_code == 200:
            with open("./img_kijiji/img_" + str(id_ad) + "_" + str(i) + ".jpg", 'wb') as f:
                f.write(img_file.content)
            i = i+1
    dettagli_row.update({"lat": lat, "lon": lon})
    dettagli.append(dettagli_row)

A Jupyter Widget

https://www.kijiji.it/annunci/cellulari-e-accessori/milano-annunci-maciachini-zara-niguarda/iphone-8-2018-nero-64gb/138705039


C:\Users\carla.molteni\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\carla.molteni\AppData\Local\Continuum\anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Milano (Milano)
https://i.ebayimg.com/00/z/vn0AAOSwwkFc6Vfq/$_59.JPG



FileNotFoundError: [Errno 2] No such file or directory: './img/img_138705039_0.jpg'

In [80]:
ds_dettagli = pd.DataFrame(dettagli)
#ds_dettagli.set_index("id")
ds_dettagli.head()

,Marca,Sito Web,Tipologia,descrizione,id_ad,lat,location,lon,nome_venditore,phone,userId
0,Apple,NaN,Cellulari,"Vendo iPhone 8 regalato a gennaio 2018, tenuto...",138705039,45.466797,Milano (Milano),9.190498,Filippo,00393408674227,26431725
1,Apple,NaN,Cellulari,"Iphone xs max da 512 gb , acquistato alla medi...",138808297,45.810681,Como (Como),9.086303,Getuar,00393347324251,26440510
2,Apple,NaN,Cellulari,"Vendo iPhone 8 regalato a gennaio 2018, tenuto...",138705039,45.466797,Milano (Milano),9.190498,Filippo,00393408674227,26431725
3,~Altro,NaN,Accessori,Vendo Bolle photo in buonissime condizioni e f...,139317911,45.937476,Cunardo (Varese),8.801245,Raffaele,N/A,18476833
4,NaN,http://tecnico telefonia,Informatica/Grafica/Web,Riparatore telefono iPhone samsug iPad tablet ...,137693462,45.156668,Mantova (Mantova),10.791719,Giuseppe,N/A,7892850


In [44]:
ds_dettagli.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 9 columns):
Comune            5 non-null object
Prezzo            4 non-null object
Tipologia         5 non-null object
date_ad           5 non-null object
descrizione       5 non-null object
id                5 non-null object
nome_venditore    5 non-null object
phone             5 non-null object
userId            5 non-null object
dtypes: object(9)
memory usage: 440.0+ bytes


In [46]:
ds_dettagli.to_csv("./kijiji_dettagli.csv")